In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

In [2]:
df = pd.read_excel('var_apl.xlsx', sheet_name=1)
df = df.set_index('Date')
df = df.sort_index()

In [3]:
conf = 99
long = False
posiciones = [100000000, 90000000]

In [4]:
def var_apl(df, posiciones, conf, long):

    # Bid y Ask
    bid_columns = [col for col in df.columns if 'Bid' in col]
    ask_columns = [col for col in df.columns if 'Ask' in col]

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))]
    df[mid_columns] = (df[bid_columns].values + df[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))]
    df[spread_columns] = (df[ask_columns].values - df[bid_columns].values) / df[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))]
    df[return_columns] = df[mid_columns].pct_change()

    # Weights
    value = posiciones * df[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    df['port_ret'] = np.dot(df[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(df['port_ret'].dropna(), 100 - conf) if long else np.percentile(df['port_ret'].dropna(), conf)
    var_cash = pv * var_pct

    # Liquidity cost
    cl_prom = df[spread_columns].mean()
    cl_estr = np.percentile(df[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.sum(cl_prom) + var_pct, np.sum(cl_estr) + var_pct

    var_apl_prom_cash, var_apl_estr_cash = np.dot(value, cl_prom) + var_cash, np.dot(value, cl_estr) + var_cash

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado'],
        'Porcentaje': [var_pct, var_apl_prom, var_apl_estr],
        'Cash': [var_cash, var_apl_prom_cash, var_apl_estr_cash]
    })

    return resultados

In [5]:
var_apl(df, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0161,"351,914,316.5583"
1,VaR Ajustado Promedio,0.0166,"357,331,030.9419"
2,VaR Ajustado Estresado,0.0172,"363,666,029.1972"
